In [261]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow import*
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mediapipe as mp

In [262]:
# Load the CSV file
df = pd.read_csv("C:\\Users\\komal\\OneDrive\\Desktop\\RAPID\\combined_output.csv")

In [263]:
# Extract features and labels
X = df.iloc[:, 1:-1].values  # Keypoints columns
y = df.iloc[:, -1].values  # Labels column

In [264]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [265]:
# Save label encoder classes
np.save('label_encoder_classes.npy', label_encoder.classes_)

In [266]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [267]:
# Reshape data to fit the CNN model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [268]:
# Define the CNN model with additional dense layers
# model = Sequential()
# model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(128, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))  # Additional dense layer
# model.add(Dense(32, activation='relu'))  # Additional dense layer
# model.add(Dropout(0.5))
# model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [269]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

C:\Users\komal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [270]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [271]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0405 - loss: 3.3302 - val_accuracy: 0.0386 - val_loss: 3.3043
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0645 - loss: 3.2903 - val_accuracy: 0.0386 - val_loss: 3.2955
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0676 - loss: 3.2701 - val_accuracy: 0.0531 - val_loss: 3.2511
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0738 - loss: 3.2342 - val_accuracy: 0.0918 - val_loss: 3.1601
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1099 - loss: 3.0916 - val_accuracy: 0.1401 - val_loss: 2.9986
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1455 - loss: 2.9875 - val_accuracy: 0.1594 - val_loss: 2.8105
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1817 - loss: 2.8070 - val_accuracy: 0.2512 - val_loss: 2.5271
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2296 - loss: 2.5903 - val_accuracy: 0.3720 - val_loss

In [272]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8314 - loss: 0.4863 
Test accuracy: 0.8405796885490417


In [273]:
# Save the model
model.save('sign_language_model.keras')

In [274]:
# Function to predict sign language word from keypoints
def predict_sign_language(keypoints):
    keypoints = np.array(keypoints).reshape(1, len(keypoints), 1)
    prediction = model.predict(keypoints)
    label_index = np.argmax(prediction)
    return label_encoder.inverse_transform([label_index])[0]

# Example prediction
example_keypoints = X_test[0].flatten()
predicted_label = predict_sign_language(example_keypoints)
print(f'Predicted label: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted label: 4
